In [1]:
!pip install mne
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch import tensor
mne.set_log_level('ERROR')
import torch
import numpy as np
from torch import nn
import scipy as sp
!pip install torch_geometric
import torch_geometric
from torch_geometric import utils
from torch_geometric.utils import dense_to_sparse
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
import torchvision
import torchvision.transforms as transforms
!pip install causal-conv1d>=1.1.0
!pip install mamba-ssm
import torch
from mamba_ssm import Mamba
!pip install gnn
import torch.nn as nn
#from SeizureSense.gnn.GraphLearner import GraphLearnerNN
from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn.conv import GATv2Conv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=323803485 sha256=cd8ee941b25398d90c135d3a180b5dc33e478c98ae4998b61749809beaff947a
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


In [2]:
class MambaBlock(nn.Module):
    def __init__(self, dim, d_state=16, d_conv=4, expand=2):
        super(MambaBlock, self).__init__()
        self.mamba = Mamba(
            d_model=dim,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        ).to("cuda")

    def forward(self, x):
        return self.mamba(x)

class StackedMambaModel(nn.Module):
    def __init__(self, num_layers, dim, d_state=16, d_conv=4, expand=2):
        super(StackedMambaModel, self).__init__()
        self.layers = nn.ModuleList([MambaBlock(dim, d_state, d_conv, expand) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class EEGMamba(nn.Module):
    def __init__(self, num_channels, num_layers, dim, d_state=16, d_conv=4, expand=2):
        super(EEGMamba, self).__init__()
        self.channel_models = nn.ModuleList([StackedMambaModel(num_layers, dim, d_state, d_conv, expand) for _ in range(num_channels)])

    def forward(self, x):
        batch_size = x.size(0)
        channel_outputs = []

        # Iterate over each channel model and corresponding channel data
        for i in range(x.size(1)):
            channel_data = x[:, i, :, :]  # Extract data for the i-th channel
            model = self.channel_models[i]
            output = model(channel_data)
            averaged_output = torch.mean(output, dim=1, keepdim=True)
            channel_outputs.append(averaged_output)

        # Stack the channel outputs along the correct dimension
        combined_output = torch.stack(channel_outputs, dim=1)
        return combined_output



In [35]:
class MambaJambaNet(nn.Module):
    def __init__(self, num_channels, num_layers, dim, d_state=16, d_conv=4, expand=2, kernel_size=2,num_classes=10):
        super(MambaJambaNet, self).__init__()
        self.mamba_model = EEGMamba(num_channels, num_layers, dim, d_state, d_conv, expand)
        self.classifer = nn.Sequential(nn.Linear(num_channels, 46),
                                       nn.ReLU(), nn.Linear(46,num_classes))

    def forward(self, x):
        print(x.shape)
        x = self.mamba_model(x)
        x = x.squeeze(2)  # making it (batch_size, num_channels, dim)
        print(x.shape)
        x=x.squeeze(2)
        print(x.shape)
        x = self.classifer(x)
        x=x.softmax(dim=1)
        return x

In [40]:
data = torch.randn(1,23,2560, 1).to("cuda")
model = MambaJambaNet(num_channels, num_layers, dim).to("cuda")
output = model(data)
print(output.shape)
print(output)

torch.Size([1, 23, 2560, 1])
torch.Size([1, 23, 1])
torch.Size([1, 23])
torch.Size([1, 10])
tensor([[0.1034, 0.1187, 0.0930, 0.0865, 0.1074, 0.0986, 0.1096, 0.0906, 0.0969,
         0.0953]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
